In [3]:
# def get_install_packages():
#   # Function to install required packages
#   !pip install -Uqqq pip --progress-bar off
#   !pip install -qqq torch==2.0.1 --progress-bar off
#   !pip install -qqq transformers==4.31.0 --progress-bar off
#   !pip install -qqq langchain==0.0.266 --progress-bar off
#   !pip install -qqq chromadb==0.4.5 --progress-bar off
#   !pip install -qqq pypdf==3.15.0 --progress-bar off
#   !pip install -qqq xformers==0.0.20 --progress-bar off
#   !pip install -qqq sentence_transformers==2.2.2 --progress-bar off
#   !pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
#   !pip install -qqq pdf2image==1.16.3 --progress-bar off
#   ! pip install auto-gptq
# get_install_packages()

In [4]:
#installing all the packages
import warnings
warnings.filterwarnings("ignore")
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
def get_pdf_file():
  # Function to get pdf files
  loader = PyPDFDirectoryLoader("pdfs")
  docs = loader.load()
  print("<<<<The Length of PDFS>>>>",len(docs))
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
  texts = text_splitter.split_documents(docs)
  print("<<<<The Length of texts>>>>",len(texts))
  print("<<<<EMBEDDING IS LOADING>>>>")
  embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE})
  db = Chroma.from_documents(texts, embeddings, persist_directory="db")
  return db

In [7]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()


In [8]:
def get_model_processed():
  #Function to load Model
  db=get_pdf_file()
  model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
  model_basename = "model"

  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

  model = AutoGPTQForCausalLM.from_quantized(
      model_name_or_path,
      revision="gptq-4bit-128g-actorder_True",
      model_basename=model_basename,
      use_safetensors=True,
      trust_remote_code=True,
      inject_fused_attention=False,
      device=DEVICE,
      quantize_config=None,
  )
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
  text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,)
  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.8})
  SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

  template = generate_prompt(
      """
  {context}

  Question: {question}
  """,
      system_prompt=SYSTEM_PROMPT,
  )
  prompt = PromptTemplate(template=template, input_variables=["context", "question"])
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # db=get_pdf_file(),
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)
  result = qa_chain("What is the per share revenue for Tesla during 2023?")
    # Extracting data into separate variables
  query = result['query']
  result1 = result['result']

  # Printing the extracted data
  print("Query:", query)
  print("\nResult:", result1)

  # return result


In [9]:
get_model_processed()

<<<<The Length of PDFS>>>> 100
<<<<The Length of texts>>>> 355
<<<<EMBEDDING IS LOADING>>>>


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

 Based on the information provided in the financial statements, the per share revenue for Tesla during 2023 cannot be determined. The financial statements provide information on total revenues and automotive sales, but do not break down the revenue figures by share. Therefore, I cannot answer the question.
Query: What is the per share revenue for Tesla during 2023?

Result:   Based on the information provided in the financial statements, the per share revenue for Tesla during 2023 cannot be determined. The financial statements provide information on total revenues and automotive sales, but do not break down the revenue figures by share. Therefore, I cannot answer the question.
